# Portfolio Optimization using PSO

This notebook implements portfolio optimization using both traditional methods and Particle Swarm Optimization (PSO).

In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

from src.data.data_processor import DataProcessor
from src.optimization.portfolio_theory import PortfolioOptimizer
from src.optimization.pso_optimizer import PSOPortfolioOptimizer

## Configuration

Set up the parameters for portfolio optimization

In [2]:
# Configuration parameters
config = {
    'data_dir': '5m',
    'n_stocks': 30,
    'risk_free_rate': 0.02,
    'objective': 'sharpe',
    'n_particles': 100,
    'iters': 200,
    'output_dir': 'results'
}

# Create output directory if it doesn't exist
if not os.path.exists(config['output_dir']):
    os.makedirs(config['output_dir'])

# Print settings
print("Portfolio Optimization Settings:")
for key, value in config.items():
    print(f"{key}: {value}")
print("\n" + "="*50 + "\n")

## Step 1: Load and Process Data

In [3]:
print("Step 1: Loading and processing stock data")
data_processor = DataProcessor(config['data_dir'])
stock_data = data_processor.load_stock_data(limit=config['n_stocks'])
daily_returns = data_processor.calculate_daily_returns()

# Calculate correlation matrix
correlation_matrix = data_processor.calculate_correlation_matrix()

# Split data into training and testing sets
train_data, test_data = data_processor.split_train_test(test_ratio=0.2)
print(f"Training data shape: {train_data.shape}")
print(f"Testing data shape: {test_data.shape}")

## Step 2: Traditional Portfolio Optimization

In [4]:
print("Step 2: Running traditional portfolio optimization")
traditional_optimizer = PortfolioOptimizer(train_data, risk_free_rate=config['risk_free_rate'])

# Calculate metrics for different portfolio strategies
weights_df, metrics_df = traditional_optimizer.portfolio_metrics_report()

print("\nPortfolio Metrics (Traditional Optimization):")
display(metrics_df)

# Save metrics and weights
metrics_df.to_csv(os.path.join(config['output_dir'], 'traditional_metrics.csv'))
weights_df.to_csv(os.path.join(config['output_dir'], 'traditional_weights.csv'))

## Step 3: PSO Portfolio Optimization

In [5]:
print("Step 3: Running PSO portfolio optimization")
pso_optimizer = PSOPortfolioOptimizer(
    train_data, 
    risk_free_rate=config['risk_free_rate'], 
    objective=config['objective']
)

pso_weights, pso_return, pso_vol, pso_sharpe = pso_optimizer.optimize(
    n_particles=config['n_particles'], 
    iters=config['iters']
)

# Save PSO weights
pd.DataFrame({
    'Asset': train_data.columns,
    'Weight': pso_weights
}).to_csv(os.path.join(config['output_dir'], 'pso_weights.csv'), index=False)

## Step 4: Compare and Visualize Results

In [6]:
print("Step 4: Comparing optimization methods")
comparison = pso_optimizer.compare_with_traditional(traditional_optimizer)
print("\nComparison with Traditional Optimization:")
display(comparison)

# Calculate portfolio performance
initial_investment = 1000
portfolio_values = {}

test_cumulative_returns = test_data.cumprod()
for name, cumulative_returns in test_cumulative_returns.items():
    portfolio_values[name] = initial_investment * (1 + cumulative_returns)

# Plot equity curve
plt.figure(figsize=(12, 8))
for name, values in portfolio_values.items():
    plt.plot(values, label=f"{name} (Final: ${values.iloc[-1]:.2f})")

plt.title(f'Portfolio Equity Curve (Initial Investment: ${initial_investment})')
plt.xlabel('Date')
plt.ylabel('Portfolio Value ($)')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.savefig(os.path.join(config['output_dir'], 'portfolio_equity_curve.png'))
plt.show()

# Display final values
final_values = {name: value.iloc[-1] for name, value in portfolio_values.items()}
final_values_df = pd.DataFrame(list(final_values.items()), 
                              columns=['Strategy', 'Final Portfolio Value ($)'])

print("\nFinal Portfolio Values (starting with $1000):")
display(final_values_df)

# Save final values
final_values_df.to_csv(os.path.join(config['output_dir'], 'final_portfolio_values.csv'), index=False)

print("\nPortfolio optimization complete!")